# RKNN Conversion Guide

----------------------------

### Before you start

Before you run the scripts or Python notebook from this project, it is recommended that you create a separate [Python virtual environment](https://docs.python.org/3/library/venv.html) so that the packages installed for the conversion process do not conflict with other packages you may already have installed.

## Preinstallation — Important Setup for Google Colab Users

This notebook requires the use of external Python scripts. Please run this snippet with the URL (GITHUB_URL) to the Photonvision repository, if it has not already been provided.

In [ ]:
GITHUB_URL = "https://github.com/boomermath/photonvision_rknn_fork/tree/f0ed73c765c935d13466639713a4331d8425e546"
SCRIPTS_FOLDER = "/scripts/rknn-convert-tool"

import sys
from urllib.parse import urlparse
import subprocess

parsed = urlparse(GITHUB_URL)

if parsed.netloc.lower() not in ['github.com', 'www.github.com']:
    print("URL must be a GitHub URL!")
    sys.exit(1)

path_parts = parsed.path.strip('/').split('/')

if len(path_parts) < 2:
    print("Invalid GitHub URL! Must have org and repo in URL")
    sys.exit(1)

org_user = path_parts[0]
repo = path_parts[1]

if not org_user or not repo:
    print("Invalid GitHub URL! Must have org and repo in URL")
    sys.exit(1)

ref = "refs/heads/main"

if len(path_parts) >= 4 and path_parts[2].lower() == "tree":
    candidate_ref = path_parts[3]
    if len(candidate_ref) == 40 and all(c in '0123456789abcdef' for c in candidate_ref.lower()):
        ref = candidate_ref
    else:
        ref = f"refs/heads/{candidate_ref}"

repo_sub_url = f"{org_user}/{repo}"
folder_url = SCRIPTS_FOLDER.strip("/")

create_onnx_raw_url = f"https://raw.githubusercontent.com/{repo_sub_url}/{ref}/{folder_url}/create_onnx.py"
auto_install_script_raw_url = f"https://raw.githubusercontent.com/{repo_sub_url}/{ref}/{folder_url}/autoinstallrknnapi.py"
create_rknn_raw_url = f"https://raw.githubusercontent.com/{repo_sub_url}/{ref}/{folder_url}/create_rknn.py"

scriptUrls = [create_onnx_raw_url, auto_install_script_raw_url, create_rknn_raw_url]

for scriptUrl in scriptUrls:
    try:
        subprocess.run(["wget", scriptUrl]).check_returncode()
        print(f"Successfully downloaded: {scriptUrl}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to download script from URL: {scriptUrl}")
        print(e)


#### *Numpy Fix*

Google Colab comes with an incompatible version of Numpy installed. To fix this, please run the following cells below and **restart your session** when prompted.

In [ ]:
%pip uninstall numpy -y
%pip install "numpy>=1.23.0,<2.0.0"

### Step 1: Convert to ONNX

To convert to ONNX, simply run the `create_onnx.py` script with your model weights as shown below.

In [ ]:
# where version is either yolov5, yolov8, or yolov11, and model_path is the path to your weights file (.pt)
%run create_onnx.py --version yolov8 --model_path weights.pt

### Step 2: Download RKNN API

You can either use a script to automatically detect and install the correct RKNN API Python library for you, or install it manually.

#### Automatic installation

Please run the script below. If it does not work, refer to the instructions for manual installation.


In [ ]:
%run autoinstallrknnapi.py

#### Manual Installation (If Automatic Installation Fails)
Visit the [RKNN Toolkit 2](https://github.com/airockchip/rknn-toolkit2) Github repository, then click on rknn-toolkit2, followed by packages.
If you are running an x86_64 CPU (e.g., most Intel and AMD processors), select that option; otherwise, choose arm64 for ARM-based computers (e.g., M-series Macs or Snapdragon processors). If you're unsure which CPU you're using, check your system settings for processor architecture information.

Once you've selected the correct CPU architecture, you'll see multiple packages. The file names will look something like:
`rknn_toolkit2-2.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl`.
The numbers after cp correspond to your Python version. For example, if you're using Python 3.10, look for a package with cp310 in the name. For Python 3.8, look for cp38; for Python 3.7, cp37, and so on.

Once you've found the correct package, click the "Raw" button to download the .whl file. Then, run the following command in your terminal, replacing rknn_toolkit2.whl with the actual path to the file you downloaded:

In [ ]:
%pip install rknn_toolkit2.whl

### Step 3: Convert to RKNN

To get started, run the `create_rknn.py` script, replacing the arguments with your own values. Refer to the table below for detailed information on each argument’s purpose and usage.

#### Overview of the `create_rknn.py` script

This script converts a YOLO ONNX model to RKNN format using a set of calibration images. It's designed to work with either:

- A flat directory of images (e.g. `train/images`), **or**
- A dataset directory containing a `data.yaml` file that defines `train`, `val`, and/or `test` folders.

##### Arguments

| Argument | Type | Description                                                                                                     |
|----------|------|-----------------------------------------------------------------------------------------------------------------|
| `--img_dir` (`-d`) | `str` (required) | Path to your image directory. This can either be a folder of images **or** a dataset folder with a `data.yaml`. |
| `--model_path` (`-m`) | `str` (required) | Path to your YOLO ONNX model, created in Step 1.                                                                |
| `--num_imgs` (`-ni`) | `int` (default: `300`) | Number of images to use for quantization calibration.                                                           |
| `--disable_quantize` (`-dq`) | `bool` (default: `False`) | Set to `True` to skip quantization entirely, not recommended for performance.                                   |
| `--rknn_output` (`-o`) | `str` (default: `out.rknn`) | File path where the final RKNN model should be saved.                                                           |
| `--img_dataset_txt` (`-ds`) | `str` (default: `imgs.txt`) | File path to store the list of images used during quantization.                                                 |
| `--verbose` (`-vb`) | `bool` (default: `False`) | Enable detailed logging from the RKNN API during conversion.                                                    |
---


##### Note

This script is designed for use with [PhotonVision](https://photonvision.org), and by default sets the target platform for RKNN conversion to `RK3588`, a chipset commonly found in many variants of the Orange Pi 5 series (e.g., Orange Pi 5, 5 Pro, 5 Plus, 5 Max, etc.). You may modify the `TARGET_PLATFORM` value in the `create_onnx.py` script to match your specific hardware or deployment requirements if necessary.

### Quantization Note

When performing quantization, it is critical to provide representative images of the objects or scenes you are trying to detect. These images are used to calibrate the model’s internal activations and greatly influence the final performance.

It is recommended to use 300–500 representative images that reflect the real-world input your model will encounter. As the old saying goes, it’s quality over quantity — having a diverse, relevant set matters more than simply having many images.

Quantization will cause some loss in model accuracy. However, if your calibration images are chosen wisely, this accuracy drop should be minimal and acceptable. If the sampled images are too uniform or unrelated, your quantized model's performance may worsen significantly.

The script will automatically sample representative images randomly from the provided dataset. While this usually works well, please verify that the dataset contains diverse and relevant examples of your target objects. As a reminder, the images used to quantize the model are stored in the text file specified by `--img_dataset_txt`.


### Optional: Download a dataset from Roboflow for quantization

Please run the code below to download a dataset from Roboflow if you do not have images available for quantization. You may replace the URL in quotes with a link to your own dataset.


In [ ]:
%wget -O roboflow.zip "https://universe.roboflow.com/ds/FaF3HbDmF7?key=iMoJR25O9H" && unzip roboflow.zip -d datasets && rm roboflow.zip

Once your dataset is prepared, run the script below to quantize and convert the ONNX model you generated in Step 1.


In [ ]:
%run create_rknn.py --img_dir ./datasets --model_path weights.onnx

And that’s it! Your RKNN model file is now ready for deployment on an Orange Pi.